# TEXT to SQL USING GOOGLE GEMINI MODEL


In [ ]:
# https://github.com/genieincodebottle/generative-ai/blob/main/genai/day-10/text_to_sql.ipynb

#Code to mount Google Drive at Colab Notebook instance
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import re
from operator import itemgetter
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_community.utilities.sql_database import SQLDatabase # Change the import to the latest in the lanchaing documentations
######## Getting the API to access gemini
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GEMINI_KEY')
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..', 'secret')))
from secret_info import google_genai_api
GOOGLE_API_KEY = google_genai_api
######## Set database for the model: 
db = SQLDatabase.from_uri('sqlite:///chinook.db') # /// = relative path, //// = absolute path
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artists LIMIT 10;")
######## Generating the model 
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)

sqlite
['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']


In [2]:
######## Ask the question
quest = "What artist has more albums?"
######## Create and execute the query 
execute_query = QuerySQLDataBaseTool(db=db)
template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the {top_k} answer.
Use the following format:
Question: "Question here"
"SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"
Only use the following tables:
{table_info}.
Provide SQL query as simple string without any markdown.
Question: {input}'''
prompt = PromptTemplate.from_template(template)
write_query = create_sql_query_chain(llm, db, prompt)
######## Format the answer
answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.
Question: {question}
{query}
SQL Result: {result}
Answer: """
)
answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)
chain.invoke({"question": quest})

'Iron Maiden has the most albums, with a total of 21 albums.'

____
____

##**Install Langchain and Google Gemini related libraries**

In [ ]:
# None of these installed as there is an error
!pip install -q -U langchain==0.1.2
!pip install -q -U google-generativeai
!pip install -q -U langchain-google-genai==1.0.3
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental

# Get Gemini API Key from Secrets
Set GEMINI_KEY secret key at Google Colab and get that here to runn Google Gemini LLM. You can get Google Gemini Key from following link https://makersuite.google.com/app/apikey

In [ ]:
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GEMINI_KEY')

import sys, os
sys.path.append(os.path.abspath(os.path.join('../..', 'secret')))
from secret_info import google_genai_api
GOOGLE_API_KEY = google_genai_api

# Loading the Data base 
Copy chinhook.db to your Google Drive

Now, chinhook.db is in our directory and we can interface with it using the SQLAlchemy-driven SQLDatabase class.

Location of chinhook.db

/content/drive/MyDrive/Colab Notebooks/chinook.db

- *I used the same file I had for the app


In [ ]:
db = SQLDatabase.from_uri('sqlite:///chinook.db') # /// = relative path, //// = absolute path
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artists LIMIT 10;")

# Chain
Let’s create a simple chain that takes a question, turns it into a SQL query, executes the query, and uses the result to answer the original question.

# Convert question to SQL query
The first step in a SQL chain or agent is to take the user input and convert it to a SQL query. LangChain comes with a built-in chain for this: create_sql_query_chain

In [ ]:
import re
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
quest = "What artist has more albums?"
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, convert_system_message_to_human=True, temperature=0.0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question":quest})

# Generate regex pattern to remove prefix and suffix
regex_pattern = r'^```sql\n|\n```$'

# Remove prefix and suffix using regex to get proper SQL query
modified_response = re.sub(regex_pattern, '', response)
print("This is the query: \n", modified_response)
db.run(modified_response)


# Execute SQL query
Now that we’ve generated a SQL query, we’ll want to execute it.

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate

execute_query = QuerySQLDataBaseTool(db=db)

template = '''Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the {top_k} answer.
Use the following format:

Question: "Question here"
"SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Provide SQL query as simple string without any markdown.

Question: {input}'''
prompt = PromptTemplate.from_template(template)

write_query = create_sql_query_chain(llm, db, prompt)
chain = write_query | execute_query
chain.invoke({"question": quest})

# Answer the question
Now that we’ve got a way to automatically generate and execute queries, we just need to combine the original question and SQL query result to generate a final answer.

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
quest = "What artist has more albums?"

answer_prompt = PromptTemplate.from_template(
"""Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
{query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": quest})

# Agents
LangChain has an SQL Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
It can answer questions that require multiple dependent queries.
It will save tokens by only considering the schema from relevant tables.
To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions

In [3]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

agent_executor = create_sql_agent(llm,
                                  toolkit=SQLDatabaseToolkit(db=db, llm=llm),
                                  agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                                  verbose=False)
agent_executor.invoke(
    {
        "input": quest
    }
)

{'input': 'What artist has more albums?', 'output': 'Iron Maiden'}